# Astrogator Object Model Walkthrough in Python API#

## Connect to STK ##

In [188]:
# Start new instance of STK using the new API
from agi.stk12.stkdesktop import STKDesktop
from agi.stk12.stkobjects import *

In [189]:
# Create a new instance of STK, grab the root, create a new scenario
stk = STKDesktop.StartApplication(visible=True) #using optional visible argument
root = stk.Root
root.NewScenario('HohmannTransferPythonAPI');

In [190]:
# Grab scenario handles
sc = root.CurrentScenario

## Create an Astrogator Satellite ##

In [191]:
# Create a new satellite
sat = sc.Children.New(AgESTKObjectType.eSatellite, 'AstrogatorSat')

In [192]:
# Set the propagator to ePropagatorAstrogator
sat.SetPropagatorType(AgEVePropagatorType.ePropagatorAstrogator)
ASTG = sat.Propagator

In [193]:
# Get a handle to the MCS
MCS = ASTG.MainSequence
type(MCS)

agi.stk12.stkobjects.astrogator.AgVAMCSSegmentCollection

## Build the MCS ## 

In [194]:
ts1 = MCS.Insert(AgEVASegmentType.eVASegmentTypeTargetSequence, 'TargetApo', '-')
type(ts1)

agi.stk12.stkobjects.astrogator.AgVAMCSTargetSequence

In [195]:
dV1 = ts1.Segments.Insert(2,'dV1','-')
type(dV1)

agi.stk12.stkobjects.astrogator.AgVAMCSManeuver

In [196]:
dV1.Maneuver.SetAttitudeControlType(AgEVAAttitudeControl.eVAAttitudeControlThrustVector)
attCon = dV1.Maneuver.AttitudeControl
type(attCon)

agi.stk12.stkobjects.astrogator.AgVAAttitudeControlImpulsiveThrustVector

In [197]:
attCon.AssignCartesian(1,0,0);

In [198]:
prop2 = ts1.Segments.Insert(AgEVASegmentType.eVASegmentTypePropagate,'PropToApo','-')

In [199]:
prop2.StoppingConditions.Add('Apoapsis')

In [200]:
ts2 = MCS.Insert(AgEVASegmentType.eVASegmentTypeTargetSequence, 'TargetCircularization', '-')

In [201]:
dV2 = ts2.Segments.InsertCopy(dV1, '-')
dV2.Properties

In [202]:
prop3 = ts2.Segments.Insert(AgEVASegmentType.eVASegmentTypePropagate, 'Prop1Day', '-')

In [203]:
scDura = prop3.StoppingConditions.Item('Duration')
print(type(scDura))

scDura.Properties.Trip = 12345

<class 'agi.stk12.stkobjects.astrogator.AgVAStoppingConditionElement'>


## Configure the Target Sequences ## 

In [204]:
# first target sequence

dV1.EnableControlParameter(AgEVAControlManeuver.eVAControlManeuverImpulsiveCartesianX)

dc1 = ts1.Profiles["Differential Corrector"] 
dc1Control = dc1.ControlParameters[0]
dc1Control.Enable = True
dc1Control.Perturbation = 0.01
dc1Control.MaxStep = 0.1

# Add results
prop2.Results.Add('R Mag')
dc1Results = dc1.Results[0]
dc1Results.Enable = True
dc1Results.DesiredValue = 42164





In [205]:
# second target sequence

# controls
dV2.EnableControlParameter(AgEVAControlManeuver.eVAControlManeuverImpulsiveCartesianX)
dV2.EnableControlParameter(AgEVAControlManeuver.eVAControlManeuverImpulsiveCartesianY)

dc2 = ts2.Profiles["Differential Corrector"] 
dc2Control0 = dc2.ControlParameters[0]
dc2Control0.Enable = True
dc2Control0.Perturbation = 0.01
dc2Control0.MaxStep = 0.1

dc2Control1 = dc2.ControlParameters[1]
dc2Control1.Enable = True
dc2Control1.Perturbation = 0.01
dc2Control1.MaxStep = 0.1


# Add results
prop3.Results.Add('Eccentricity')
dc2Results0 = dc2.Results[0]
dc2Results0.Enable = True
dc2Results0.DesiredValue = 0.0

prop3.Results.Add('Inclination')
dc2Results1 = dc2.Results[1]
dc2Results1.Enable = True
dc2Results1.DesiredValue = 0.0




In [206]:
# Set the Target Profiles Action to Run active profiles 
ts1.Action = AgEVATargetSeqAction.eVATargetSeqActionRunActiveProfiles
ts2.Action = AgEVATargetSeqAction.eVATargetSeqActionRunActiveProfiles

## Run the MCS ##

In [207]:
ASTG.RunMCS()

In [208]:
type(dV1.Results)

agi.stk12.stkobjects.astrogator.AgVACalcObjectCollection